### Approach

In [1]:
                                                                                        import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np
import nlpaug.augmenter.word as naw # To synonym, see  to more info https://github.com/makcedward/nlpaug
import nlpaug.augmenter.char as nac
from mtranslate import translate

C:\Users\nahue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gpt_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_nawel.csv", sep = ';')
gpt_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_nawel.csv", sep = ';')
gpt_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_nawel.csv", sep = ';')
gpt_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_nawel.csv", sep = ';')

In [3]:
# Con Bert
aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', aug_p=0.1)

In [4]:
# Wordnet - spa
aug1 = naw.SynonymAug(aug_src='wordnet', lang='spa')

In [5]:
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

In [6]:
# 2. Replace words with synonyms
# Example for values in each sentences
print('Asistir')
print(hand_nawel_0.text[10])
print(f'Bert: {aug.augment(hand_nawel_0.text[10])}') # show secoond row
print(f'Wordnet: {aug1.augment(hand_nawel_0.text[10])} \n')

print('No Asistir')
print(hand_nawel_1.text[10])
print(f'Bert: {aug.augment(hand_nawel_1.text[10])}') # show secoond row
print(f'Wordnet: {aug1.augment(hand_nawel_1.text[10])} \n')

print('Reagendar')
print(hand_nawel_2.text[10])
print(f'Bert: {aug.augment(hand_nawel_2.text[10])}') # show secoond row
print(f'Wordnet: {aug1.augment(hand_nawel_2.text[10])} \n')

print('Pedir')
print(hand_nawel_3.text[10])
print(f'Bert: {aug.augment(hand_nawel_3.text[10])}') # show secoond row
print(f'Wordnet: {aug1.augment(hand_nawel_3.text[10])} \n')

Asistir
Por supuesto
Bert: ['el supuesto']
Wordnet: ['Por supuesto'] 

No Asistir
No me siento preparado para rendir la prueba, cancelame la hora
Bert: ['yo me siento preparado para rendir la prueba, pero la hora']
Wordnet: ['No me siento preparado para rendir la test, cancelame la hora'] 

Reagendar
Puedes ayudarme a cambiar la hora para la licencia
Bert: ['puedes ayudarme a cambiar la vida para la revolucion']
Wordnet: ['Puedes ayudarme a cambiar la hora para la licencia'] 

Pedir
Hola, ¿acá puedo pedir la hora para la licencia?
Bert: ['hola, ¿ donde puedo pedir la hora para la virgen?']
Wordnet: ['Hola, ¿ acá puedo pedir la esponsales para la licencia?'] 



In [10]:
aug_key = nac.KeyboardAug() #Keyboard Augmenter: Substitute character by keyboard distance
aug_ins = nac.RandomCharAug(action="insert") # insert randomw characters
aug_subs = nac.RandomCharAug(action="substitute") # Substitute character randomly
aug_swp = nac.RandomCharAug(action="swap") # swap randomly
aug_del = nac.RandomCharAug(action="delete") # delete randomly
#aug_mis = naw.SpellingAug() # Substitute word by spelling mistake words dictionary (inly english)

In [11]:
# 3. add noise

# Example for values in each sentences

print('Asistir')
print(hand_nawel_0.text[10])
print(f'Keyboard: {aug_key.augment(hand_nawel_0.text[10])}') # show secoond row
print(f'Random Insert: {aug_ins.augment(hand_nawel_0.text[10])}') # show secoond row
print(f'Random Substitution: {aug_subs.augment(hand_nawel_0.text[10])}') # show secoond row
print(f'Random Swap: {aug_swp.augment(hand_nawel_0.text[10])}') # show secoond row
print(f'Random delete: {aug_del.augment(hand_nawel_0.text[10])} \n') # show secoond row

print('No Asistir')
print(hand_nawel_1.text[10])
print(f'Keyboard: {aug_key.augment(hand_nawel_1.text[10])}') # show secoond row
print(f'Random Insert: {aug_ins.augment(hand_nawel_1.text[10])}') # show secoond row
print(f'Random Substitution: {aug_subs.augment(hand_nawel_1.text[10])}') # show secoond row
print(f'Random Swap: {aug_swp.augment(hand_nawel_1.text[10])}') # show secoond row
print(f'Random delete: {aug_del.augment(hand_nawel_1.text[10])}  \n') # show secoond row

print('Reagendar')
print(hand_nawel_2.text[10])
print(f'Keyboard: {aug_key.augment(hand_nawel_2.text[10])}') # show secoond row
print(f'Random Insert: {aug_ins.augment(hand_nawel_2.text[10])}') # show secoond row
print(f'Random Substitution: {aug_subs.augment(hand_nawel_2.text[10])}') # show secoond row
print(f'Random Swap: {aug_swp.augment(hand_nawel_2.text[10])}') # show secoond row
print(f'Random delete: {aug_del.augment(hand_nawel_2.text[10])}  \n') # show secoond row

print('Pedir')
print(hand_nawel_3.text[10])
print(f'Keyboard: {aug_key.augment(hand_nawel_3.text[10])}') # show secoond row
print(f'Random Insert: {aug_ins.augment(hand_nawel_3.text[10])}') # show secoond row
print(f'Random Substitution: {aug_subs.augment(hand_nawel_3.text[10])}') # show secoond row
print(f'Random Swap: {aug_swp.augment(hand_nawel_3.text[10])}') # show secoond row
print(f'Random delete: {aug_del.augment(hand_nawel_3.text[10])}  \n') # show secoond row


Asistir
Por supuesto
Keyboard: ['Por Wupuexro']
Random Insert: ['Por svutpuestho']
Random Substitution: ['Por 3upsysto']
Random Swap: ['Por uspuesto']
Random delete: ['Por suuso'] 

No Asistir
No me siento preparado para rendir la prueba, cancelame la hora
Keyboard: ['No me siento prepArxdk pz5a rendir la peuebQ, sancw/ame la hora']
Random Insert: ['No me siento Tpr(eparad@o poar(a OrenXdir la pruOebBa, cancelame la hora']
Random Substitution: ['No me siento pre$araqm para r7nd6r la pmuoba, cancelame la borU']
Random Swap: ['No me siento preparado para rendir la rpueab, accenlame la ohar']
Random delete: ['No me siento reprdo para reni la ruba, canlae la hora']  

Reagendar
Puedes ayudarme a cambiar la hora para la licencia
Keyboard: ['LuedeA aHudaGmS a caNniWr la hora para la licencia']
Random Insert: ['Puhesdes ayudarme a ccaHmb%iar la hOoxra para la licencia']
Random Substitution: ['#uede0 ayudarme a cambiar la hOr5 Q3ra la licencia']
Random Swap: ['Upedse ayudarme a cambiar la hrao

In [12]:
# Extra : Translate Ideas
def sp_to_en(data):
    augmented_data = []
    for sentence in data:
        # Translate Spanish sentence to English
        translated_sentence = translate(sentence, "en")
        augmented_data.append(translated_sentence)

    return augmented_data

def en_to_sp(data):
    augmented_data = []
    for sentence in data:
        # Translate English sentence to Spanish
        translated_sentence = translate(sentence, "es")

        augmented_data.append(translated_sentence)

    return augmented_data

Other option, translate to english before the data aug and after that, translate to spanish.

In [15]:
# This idea can be used to find synonyms in english.
original = hand_nawel_0.text.tolist()
augmented_sentences = sp_to_en(original)
augmented_sentences2 = en_to_sp(augmented_sentences)

print(f'Orig: {original[1]}')
print(f'Eng: {augmented_sentences[1]}')
print(f'Spa: {augmented_sentences2[1]} \n')

Orig: si, confirmo
Eng: yes, I confirm
Spa: si, lo confirmo 



In [16]:
print(augmented_sentences[:3])

['Hello, I confirm my attendance', 'yes, I confirm', 'Yes, I will attend the stipulated date']


In [17]:
aug_wordnet = naw.SynonymAug(aug_src='wordnet')

In [18]:
aug_fb = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

In [20]:
# In this case, we only can use the synonm augmenter
# Substite by w2v glove or fasttext
augmented_text_fb = aug_fb.augment(augmented_sentences)
augmented_text_wn = aug_wordnet.augment(augmented_sentences)
translates_fb = en_to_sp(augmented_text_fb)
translates_wn = en_to_sp(augmented_text_wn)

print('######################Casos WORDNET######################')
for i in range(len(augmented_text_fb[:5])):
    print(f"Original: {original[i]}")
    print(f"English: {augmented_sentences[i]}")
    print(f"Augmented Text: {augmented_text_wn[i]}")
    print(f"To spanish version: {translates_wn[i]} \n")

print('######################Casos Facebook######################')
for i in range(len(augmented_text_fb[:5])):
    print(f"Original: {original[i]}")
    print(f"English: {augmented_sentences[i]}")
    print(f"Augmented Text: {augmented_text_fb[i]}")
    print(f"To spanish version: {translates_fb[i]} \n")

######################Casos WORDNET######################
Original: Hola, confirmo mi asistencia
English: Hello, I confirm my attendance
Augmented Text: Hello, I confirm my attending
To spanish version: Hola confirmo mi asistencia 

Original: si, confirmo
English: yes, I confirm
Augmented Text: yes, I affirm
To spanish version: si, afirmo 

Original: Si, asistiré a la fecha estipulada
English: Yes, I will attend the stipulated date
Augmented Text: Yes, Iodin testament attend the stipulated day of the month
To spanish version: Si, Iodin testament asistir el día estipulado del mes 

Original: Si
English: And
Augmented Text: And
To spanish version: Y 

Original: Ok
English: Ok
Augmented Text: Oklahoma
To spanish version: Oklahoma 

######################Casos Facebook######################
Original: Hola, confirmo mi asistencia
English: Hello, I confirm my attendance
Augmented Text: Hello, I confirm my participation
To spanish version: Hola confirmo mi participación 

Original: si, confir